In [1]:
!pip install matplotlib_inline

In [1]:
import os

import joblib as jb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torchvision
from d2l import torch as d2l
from torch import nn
from torchmetrics import R2Score
from torchsummary import summary

In [2]:
class Flatten(nn.Module):
    """Converts N-dimensional tensor into 'flat' one."""

    def __init__(self, keep_batch_dim=True):
        super().__init__()
        self.keep_batch_dim = keep_batch_dim

    def forward(self, x):
        if self.keep_batch_dim:
            return x.view(x.size(0), -1)
        return x.view(-1)

In [3]:
class RNNRegressor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(RNNRegressor, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

In [4]:
# Calculating the loss
# loss = nn.CrossEntropyLoss(reduction='none')
loss = nn.MSELoss(reduction='none')

def evaluate_loss(data_iter, net, devices):
    l_sum, n = 0.0, 0
    for features, labels in data_iter:
        features, labels = features.to(devices[0]), labels.to(devices[0])
        outputs = net(features)
        l = loss(outputs, labels)
        l_sum += l.sum()
        n += labels.numel()
    return l_sum / n

In [5]:
def loss_plotter(train_losses,test_losses):
    plt.figure(figsize = (10,6) )
    plt.plot(train_losses, 'r*-', label = "Train Loss")
    plt.plot(test_losses, 'b*-', label = "Test Loss")
    plt.legend()
    plt.grid()
    plt.xlabel("Epochs")
    plt.ylabel("The Loss Value")
    

In [6]:
spo2_vals = jb.load('strided_dataset_spo2_vals.pk')

In [7]:
stride_dataset = jb.load('strided_dataset.pk')

In [8]:
dataset = pd.DataFrame.from_dict(stride_dataset).transpose()

In [9]:
dataset_spo2 = pd.DataFrame.from_dict(spo2_vals).transpose()

In [10]:
dataset_spo2 = dataset_spo2/100

In [11]:
X_train, X_valid, X_test = np.split(dataset.sample(
    frac=1), [int(.8*len(dataset)), int(.9*len(dataset))])
y_train, y_valid, y_test = dataset_spo2.loc[X_train.index], dataset_spo2.loc[
    X_valid.index], dataset_spo2.loc[X_test.index]

In [12]:

# unroll the data
X_train_unrolled = []
y_train_unrolled = []
for r in X_train.index:
    for c in X_train.columns:
        # skip values containing NaN
        if not np.isnan(X_train.loc[r][c]).any() and not np.isnan(y_train.loc[r][c]):
            X_train_unrolled.append(X_train.loc[r][c])
            y_train_unrolled.append(y_train.loc[r][c])

X_valid_unrolled = []
y_valid_unrolled = []
for r in X_valid.index:
    for c in X_valid.columns:
        # skip values containing NaN
        if not np.isnan(X_valid.loc[r][c]).any() and not np.isnan(y_valid.loc[r][c]):
            X_valid_unrolled.append(X_valid.loc[r][c])
            y_valid_unrolled.append(y_valid.loc[r][c])

X_test_unrolled = []
y_test_unrolled = []
for r in X_test.index:
    for c in X_test.columns:
        # skip values containing NaN
        if not np.isnan(X_test.loc[r][c]).any() and not np.isnan(y_test.loc[r][c]):
            X_test_unrolled.append(X_test.loc[r][c])
            y_test_unrolled.append(y_test.loc[r][c])

In [13]:
# convert to tensors
X_train, y_train = torch.tensor(X_train_unrolled, dtype=torch.float32), torch.tensor(
    y_train_unrolled, dtype=torch.float32)
X_valid, y_valid = torch.tensor(X_valid_unrolled, dtype=torch.float32), torch.tensor(
    y_valid_unrolled, dtype=torch.float32)
X_test, y_test = torch.tensor(X_test_unrolled, dtype=torch.float32), torch.tensor(
    y_test_unrolled, dtype=torch.float32)

C:\Users\RASIME~1\AppData\Local\Temp/ipykernel_18836/997760739.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:233.)
  X_train, y_train = torch.tensor(X_train_unrolled, dtype=torch.float32), torch.tensor(


In [14]:
# reshape y tensors
y_train = y_train.reshape(-1, 1)
y_valid = y_valid.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [15]:
# normalize x data
X_train_mean = X_train.mean() 
X_train_std = X_train.std()
X_train = (X_train - X_train_mean) / (X_train_std + 1e-7)

X_valid_mean = X_valid.mean()
X_valid_std = X_valid.std()
X_valid = (X_valid - X_valid_mean) / (X_valid_std + 1e-7)

X_test_mean = X_test.mean()
X_test_std = X_test.std()
X_test = (X_test - X_test_mean) / (X_test_std + 1e-7)

In [16]:
batch_size = 64
train_iter = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(
    X_train, y_train), batch_size=batch_size, shuffle=True)
valid_iter = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(
    X_valid, y_valid), batch_size=batch_size, shuffle=True)
test_iter = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(
    X_test, y_test), batch_size=batch_size, shuffle=True)

In [17]:
num_epochs = 10
lr, wd = 0.001, 1e-4
lr_period, lr_decay = 2, 0.9

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
class RNNRegressor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size = 1):
        super(RNNRegressor, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, 32)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(32,16)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(16,output_size)
        

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(1, x.size(0), self.hidden_size)
        
        # Forward propagate the RNN
        out, _ = self.rnn(x, h0)
        
        # Pass the output of the final time step through the fully-connected layer
        out = self.fc(out[:, -1, :])
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out


In [20]:
model = RNNRegressor(input_size=240, hidden_size=128).to(device)
#Setting the loss function
cost = nn.L1Loss()

#Setting the optimizer with the model parameters and learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

#this is defined to print how many steps are remaining when training
total_step = len(train_iter)

In [21]:
summary(model, input_size = (1,240))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
               RNN-1  [[-1, 1, 128], [-1, 2, 128]]               0
            Linear-2                   [-1, 32]           4,128
              ReLU-3                   [-1, 32]               0
            Linear-4                   [-1, 16]             528
              ReLU-5                   [-1, 16]               0
            Linear-6                    [-1, 1]              17
Total params: 4,673
Trainable params: 4,673
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.25
Params size (MB): 0.02
Estimated Total Size (MB): 0.27
----------------------------------------------------------------


In [22]:
train_losses = []
test_losses = []

num_epochs = 100
for epoch in range(num_epochs):
    

    
    
    for i, (features, labels) in enumerate(train_iter):
        batch_size = features.shape[0]
        features = torch.reshape(features, (batch_size,1,240))
        labels = torch.reshape(labels, (batch_size,1,1))
        features = features.to(device)
        labels = labels.to(device)
        outputs = model(features)
        loss = cost(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        
        
    for i, (features_test, labels_test) in enumerate(test_iter):
        batch_size = features_test.shape[0]
        features_test = torch.reshape(features_test, (batch_size,1,240))
        labels_test = torch.reshape(labels_test, (batch_size,1,1))
        
        features_test = features_test.to(device)
        labels_test = labels_test.to(device)
        outputs_test = model(features_test)
        loss_test = cost(outputs_test,labels_test)
     
    train_losses.append(loss.item()*100)
    test_losses.append(loss_test.item()*100)
    print("1) The Train Loss value with the epoch: " + str(epoch) + " is equal to " + str(loss.item()*100))
    print("2) The Test Loss value with the epoch: " + str(epoch) + " is equal to " + str(loss_test.item()*100))
    print("-------------------------------------------------------------------")
    
    
    if epoch == num_epochs-2:
        jb.dump(train_losses,"train_losses.pk")
        jb.dump(test_losses,"test_losses.pk")
        break



C:\Users\rasimemirdagi\anaconda3\lib\site-packages\torch\nn\modules\loss.py:101: UserWarning: Using a target size (torch.Size([64, 1, 1])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
C:\Users\rasimemirdagi\anaconda3\lib\site-packages\torch\nn\modules\loss.py:101: UserWarning: Using a target size (torch.Size([48, 1, 1])) that is different to the input size (torch.Size([48, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
C:\Users\rasimemirdagi\anaconda3\lib\site-packages\torch\nn\modules\loss.py:101: UserWarning: Using a target size (torch.Size([16, 1, 1])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure th

1) The Train Loss value with the epoch: 0 is equal to 1.9990041851997375
2) The Test Loss value with the epoch: 0 is equal to 1.8646789714694023
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 1 is equal to 2.1735893562436104
2) The Test Loss value with the epoch: 1 is equal to 1.711743324995041
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 2 is equal to 2.0317507907748222
2) The Test Loss value with the epoch: 2 is equal to 1.7866887152194977
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 3 is equal to 1.7215089872479439
2) The Test Loss value with the epoch: 3 is equal to 1.798241212964058
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 4 is equal to 2.108648419380188
2) The Test Loss value with the epoch: 4 is equal to 1.974502019584179
-------

1) The Train Loss value with the epoch: 39 is equal to 1.708332821726799
2) The Test Loss value with the epoch: 39 is equal to 2.197059988975525
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 40 is equal to 2.0389191806316376
2) The Test Loss value with the epoch: 40 is equal to 1.7449039965867996
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 41 is equal to 1.7160682007670403
2) The Test Loss value with the epoch: 41 is equal to 1.7965646460652351
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 42 is equal to 1.8490420654416084
2) The Test Loss value with the epoch: 42 is equal to 1.6670199111104012
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 43 is equal to 1.9494781270623207
2) The Test Loss value with the epoch: 43 is equal to 1.370778493583

1) The Train Loss value with the epoch: 78 is equal to 2.106582373380661
2) The Test Loss value with the epoch: 78 is equal to 1.7577225342392921
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 79 is equal to 2.041666768491268
2) The Test Loss value with the epoch: 79 is equal to 1.4112519100308418
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 80 is equal to 2.106584422290325
2) The Test Loss value with the epoch: 80 is equal to 2.3840170353651047
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 81 is equal to 1.762544922530651
2) The Test Loss value with the epoch: 81 is equal to 1.3622093014419079
-------------------------------------------------------------------
1) The Train Loss value with the epoch: 82 is equal to 2.0090393722057343
2) The Test Loss value with the epoch: 82 is equal to 1.56249981373548